<a href="https://colab.research.google.com/github/NongNNew/Project_499/blob/main/Project_499_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

In [1]:
!git clone https://github.com/NongNNew/Project_499.git

Cloning into 'Project_499'...
remote: Enumerating objects: 1390, done.
remote: Counting objects: 100% (558/558), done.
remote: Compressing objects: 100% (550/550), done.
remote: Total 1390 (delta 42), reused 520 (delta 8), pack-reused 832
Receiving objects: 100% (1390/1390), 232.51 MiB | 20.68 MiB/s, done.
Resolving deltas: 100% (119/119), done.
Checking out files: 100% (1122/1122), done.


In [2]:
fruit_veget = ['กระชาย','กระท้อน','กระเทียม1','กระเทียม2','กระเพรา','กล้วยน้ำว้า','กล้วยหอม',
               'ข้าวโพด','ไข่น้ำ',
               'ตะขบไทย','ตะไคร้',
               'ถั่วฝักยาว','ถั่วลันเตา','ถั่วลิสง',
               'ทับทิม',
               'น้อยหน่า','น้ำเต้า',
               'ผักกระเฉด','ผักกุยช่าย','ผักขึ้นช่าย','ผักชะอม','ผักชี','ผักชีฝรั่ง','ผักตำลึง',
               'มะปราง','มะพลับ','มะละกอ','มะกรูด','มะเขือพวง','มะเขือเทศ','มะระ','มะรุม','มะตูม','มันแกว1','มันแกว2','มันเทศ','มันฝรั่ง','มันสำปะหลัง',
               'บวบ','ใบชะพลู1','ใบชะพลู2','ใบบัวบก','ใบแมงลัก','ใบยอ',
               'พริกขี้หนู','พริกสด','พุทรา','เพกา',
               'ฝรั่ง',
               'ฟัก','ฟักทอง',
               'สับปะรด','สะเดา','สะระแหน่',
               'หัวหอม1','หัวหอม2']

source = {'audio_time':[],
          'sampling_rate':[],
          'feature_extraction':[],
          'label':[],
          'description':[]}

In [3]:
import pandas as pd
from glob import glob
import librosa

for types in fruit_veget:
    audio_files = glob('/content/Project_499/Record_audio/' + str(types) + '/*.wav')

    for audio in audio_files:
        y,sr = librosa.load(audio,duration=5,offset=0)
        source['description'].append(str(types))
        source['audio_time'].append(y)
        source['sampling_rate'].append(sr)
        source['feature_extraction'].append(librosa.feature.mfcc(y,sr,n_mfcc=13))
        source['label'].append(fruit_veget.index(types))

In [4]:
df = pd.DataFrame.from_dict(source)
df

,audio_time,sampling_rate,feature_extraction,label,description
0,"[-0.00010681152, -0.00010681152, -0.0001525878...",22050,"[[-622.78125, -623.396, -623.63983, -623.7701,...",0,กระชาย
1,"[-7.6293945e-05, -0.00015258789, -0.0001068115...",22050,"[[-670.6008, -670.93756, -673.4263, -673.5513,...",0,กระชาย
2,"[-0.0001373291, -0.00010681152, -0.0001373291,...",22050,"[[-641.21436, -640.2155, -639.54114, -638.4883...",0,กระชาย
3,"[-9.1552734e-05, -0.00012207031, -0.0001373291...",22050,"[[-644.627, -645.0809, -645.80524, -646.2903, ...",0,กระชาย
4,"[1.5258789e-05, -3.0517578e-05, 4.5776367e-05,...",22050,"[[-624.1429, -624.4267, -624.8286, -624.58374,...",0,กระชาย
...,...,...,...,...,...
1115,"[3.0517578e-05, -6.1035156e-05, 6.1035156e-05,...",22050,"[[-620.5758, -620.8986, -620.76514, -619.66296...",55,หัวหอม2
1116,"[-7.6293945e-05, -9.1552734e-05, 0.0, 1.525878...",22050,"[[-617.5053, -617.5226, -617.99506, -618.711, ...",55,หัวหอม2
1117,"[-7.6293945e-05, -3.0517578e-05, 4.5776367e-05...",22050,"[[-607.6926, -606.95215, -607.1966, -606.29114...",55,หัวหอม2
1118,"[0.0, 0.0, -3.0517578e-05, 3.0517578e-05, -3.0...",22050,"[[-644.77423, -645.56177, -644.1022, -643.1271...",55,หัวหอม2


In [12]:
import numpy as np
import pandas as pd

Test = pd.DataFrame(columns=source.keys())

# create Test model
for description in np.unique(df['description']):
    imp = df[df['description']==description].sample(5,random_state=1)
    Test = pd.concat([Test,imp], axis=0)

# create Train model
Train = df.drop(Test.index)

X_train, X_test = np.array(Train['feature_extraction'].to_list()), np.array(Test['feature_extraction'].to_list())
y_train, y_test = np.array(Train['label'].to_list()), np.array(Test['label'].to_list())

In [13]:
import tensorflow as tf
from tensorflow import keras 
np.random.seed(42)
tf.random.set_seed(42)

model = keras.Sequential()
model.add(keras.layers.Conv2D(128, 5, input_shape=(X_train.shape[1], X_train.shape[2], 1), activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(128, 5, activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(len(np.unique(df['label'])), activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train[:,:,:,None], y_train, epochs=100)

Epoch 1/100
27/27 [==============================] - 1s 31ms/step - loss: 17.6381 - accuracy: 0.0143
Epoch 2/100
27/27 [==============================] - 1s 26ms/step - loss: 4.0096 - accuracy: 0.0274
Epoch 3/100
27/27 [==============================] - 1s 26ms/step - loss: 3.9405 - accuracy: 0.0345
Epoch 4/100
27/27 [==============================] - 1s 27ms/step - loss: 3.7675 - accuracy: 0.0702
Epoch 5/100
27/27 [==============================] - 1s 26ms/step - loss: 3.3554 - accuracy: 0.1464
Epoch 6/100
27/27 [==============================] - 1s 27ms/step - loss: 2.6549 - accuracy: 0.2833
Epoch 7/100
27/27 [==============================] - 1s 27ms/step - loss: 2.1510 - accuracy: 0.3857
Epoch 8/100
27/27 [==============================] - 1s 26ms/step - loss: 1.8063 - accuracy: 0.5060
Epoch 9/100
27/27 [==============================] - 1s 26ms/step - loss: 1.4070 - accuracy: 0.5905
Epoch 10/100
27/27 [==============================] - 1s 27ms/step - loss: 1.0677 - accuracy: 0.692

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 9, 212, 128)       3328      
_________________________________________________________________
dropout_3 (Dropout)          (None, 9, 212, 128)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 106, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 106, 128)       409728    
_________________________________________________________________
dropout_4 (Dropout)          (None, 4, 106, 128)       0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 53, 128)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 13568)            

In [15]:
model.evaluate(X_test[:,:,:,None],y_test)

9/9 [==============================] - 0s 10ms/step - loss: 0.2235 - accuracy: 0.9464


[0.22349931299686432, 0.9464285969734192]

In [16]:
from sklearn.metrics import classification_report

predicted_classes = np.argmax(np.round(model.predict(X_test[:,:,:,None])),axis=1)
target_names = [f"Class {label}" for label in range(len(np.unique(df['label'])))]

print(classification_report(y_test, predicted_classes, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.83      1.00      0.91         5
     Class 1       1.00      0.80      0.89         5
     Class 2       0.83      1.00      0.91         5
     Class 3       1.00      0.20      0.33         5
     Class 4       1.00      1.00      1.00         5
     Class 5       0.83      1.00      0.91         5
     Class 6       1.00      1.00      1.00         5
     Class 7       1.00      1.00      1.00         5
     Class 8       1.00      1.00      1.00         5
     Class 9       1.00      0.80      0.89         5
    Class 10       1.00      1.00      1.00         5
    Class 11       1.00      1.00      1.00         5
    Class 12       1.00      1.00      1.00         5
    Class 13       0.83      1.00      0.91         5
    Class 14       1.00      1.00      1.00         5
    Class 15       0.83      1.00      0.91         5
    Class 16       1.00      0.80      0.89         5
    Class 17       1.00    